# SIXT33N Project
## Phase 1: PCA - Genre Sorting

### EE 16B: Designing Information Devices and Systems II, Spring 2016

Written by Nathaniel Mailoa and Emily Naviasky (2016)

nmailoa@berkeley.edu &emsp; enaviasky@berkeley.edu

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-

## Table of Contents

* [Introduction](#intro)
* [Part 1: Data Collection](#part1)
* [Part 2: Principal Component Analysis](#part2)
* [Part 3: Classification](#part3)
* [Part 4: Launchpad Implementation](#part4)

<a id='intro'></a>
## Introduction

In this version of the project, SIXT33N is a very picky robot when it comes to music. There is one genre of music that SIXT33N loves and will chase, but there is another genre that SIXT33N runs away from. Which genres are those? That is what you are going to discover in this phase.

For phase 1, you will develop the PCA classifier that allows SIXT33N to tell the difference between the two genres. You will examine several different musical genres, and determine which ones will be easiest to sort by PCA. However, your teammates are just starting to build the microphone circuit that will serve as SIXT33N's ears, so you will acquire sample data using the class microphone board. (You just need enough data to experiment with, you will collect your final set of training data in the next phase.)

Once you have some sample data collected, you will develop a model in this iPython Notebook (since iPython has pretty graphs and lots of computing power, and a Launchpad does not). You will look at that sample data to get an idea of what features might be easily distingused, and which of the several genres you have collected will be the easiest to separate. Then, you can experiment with different methods of signal processing to accentuate those differences. By the end you will hace developed a classifier algorithm. 

Once you (and your GSI) are satisfied with the classifier's accuracy, you will port the classifier into the Launchpad code in Energia. However, since your Launchpad has so much less computing power than your PC, you will examine ways to optimize the classification before you port it.  

As a reminder, you will have 2 weeks to work on this phase and choose the genres and the analysis methods that work well for you before you collect your final set of training data on the microphone circuit that your teammates are building.

The goals of this phase are as follows:
- Experiment with different genres
- PCA + Classifier (2 genres)
- Check accuracy
- PCA projection on Launchpad

As mentioned in the main project page, there is a checkpoint every week. Each checkpoint is worth 5 points, and if you are late you are awarded 4 points. The checkpoints are due in the beginning of the lab in the week after. For this phase, the checkpoints (marked **<span style="color:green">green</span>** in the Notebook) are:
- Week 1: First pass through PCA with sample data; GSI feedback
- Week 2: Classification target met in Python

<a id='part1'></a>
## <span style="color:blue">Part 1: Data Collection</span>

### Materials
- Microphone front-end circuit
- Launchpad + USB

SIXT33N, being a robot, does not listen to music quite the same way that we do. While we listen to music temporally, SIXT33N much prefers to take the DFT of samples of the music and judge them based on their frequency content.

Keeping in mind that you are distinguishing music in the frequency domain, brainstorm with your partner. What genres do you think will look the most different? Pick four or more music genres that you think will classify well.

**<span style="color:red">What genres are you going to try? Why?</span>** 

Firstly, upload the sketch <b>`collect-data-music.ino`</b> to your Launchpad. This sketch gathers ADC samples every 0.35ms, performs a 128-point FFT, and streams the real and imaginary parts of the coefficient back to the PC. This code also sets some pins to input pins, which is required for the microphone board to work.

Attach the class microphone board to your Launchpad, as shown below. Make sure that you are connecting row 1 on the mic board to 3.3V/5V on the Launchpad. Next, you will want to remove the jumper on your Launchpad labeled 5V. You are doing this so that you can power the mic-board's 5V power off of your bench power supply. The power supply is a quieter power source than USB. <b>Put the jumper so that it is connected only to the pin closest to the usb, so that you do not lose it. We do not have replacements for them.</b> Make sure that your power supply is delivering a 100mA current limited 5V. You can connect 5V to either the pin of the jumper that you just removed closest to the green board, or to the bottom pin labeled 5V and ground to the ground pin in the opposite bottom corner. Make sure everything is hooked up as below.

<center>
<img width="600px" src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/proj-micboard.png">
</center>

Probe the test point of the board using your oscilloscope. You should see a changing signal centered around 1.5V. If the signal is centered too far from 1.5V, take a screw driver and turn the potentiometer on the microphone board until the signal is in the right place again. Make a noise at the microphone; you should see the signal change to reflect the sound you just made. If you are close enough or loud enough, you should be able to get the peak-to-peak amplitude of your signal all the way up to 3V.

<center>
<img width="400px" src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/proj-waveform.png">
</center>

Now, go track down one of the genres you decided on using. Play the music to the microphone, and see how it looks on the scope. Move the speaker towards and away from the microphone. Notice how the signal gets distorted when the sound is too loud? Good audio data is loud enough that noises from the rest of the room don't show up too much, but quiet enough that the signal doesn't get distorted when it saturates the output. Position your speakers so that the music you want to collect is loud but not distorted.

The next step is to sample that signal and save it to a file on your computer so you can start experimenting with it. You should already have <b>`collect-data-music.ino`</b> uploaded. As a reminder, this sketch gathers ADC samples every 0.35ms, performs a 128-point FFT, and streams the real and imaginary parts of the coefficient back to the PC. With the music you selected playing, open a terminal and run the script:

<b>`python collect-data.py log.csv`</b>

<b>After the code is uploaded and the script is run, hit the reset button.</b> The Launchpad is sending the real and imaginary parts of the FFT coefficients to the computer and this script is collectin them and storing them in `log.csv`. After about thirty seconds of data collection, stop the script by hitting `Ctrl-C`. Check the `log.csv` and make sure that it looks like an FFT and is not just full of zeros. It might help to plot the data to make sure.

This little snippet that you collected is too short to do anything useful with, but you now know the process for collecting data. You can now begin collecting your sample data. 

Keep in mind that for each genre, you want around two or three minutes worth of sample data. You will collect more once you settle on the two genres that you want to use and once your partners are finished building the mic circuit that you will use. It is important that your final data is collected on your final mic circuit or else PCA will not work as well. 

For each genre, once the data is collected, rename `log.csv` to something descriptive of the genre you just played. Again, do this for each genre or else the script will over write the data you just collected.  

####For your Consideration:

Once you have your four or more genres collected, you can move onto the PCA classification and data analysis below. You may realize in the next section that none of your inital choices of genre are sorting quite as well as you would like. Don't be afraid to come back to this section and try collecting data from different genres using what you have learned about what makes a genre sortable. However, keep in mind that signal processing will play a big part in making a genre easy to sort.

That is part of the design aspect of this phase, deciding when your raw data is not good enough and when your data processing is not good enough. Feel free to discuss your design choices with your TA at any time!

<a id='part2'></a>
## <span style="color:blue">Part 2: Principal Component Analysis</span>

In the previous part you decided on genres to test and collected all the data that you will need to start experimenting. Now the fun can really start! 

The python function `read_csv()` is provided below to extract the data from the the .csv files you just collected. Call `read_csv()` on each genre file, and name each result a different variable.

In [1]:
import numpy as np
import numpy.matlib
import scipy.io
import matplotlib.pyplot as plt
import csv
%matplotlib inline

In [ ]:
def read_csv(filename):
    """
    Reads a csv file and returns the first 20 recordings from the file
    Input:
        filename: csv filename
    Output:
        data: a 20x66 matrix corresponding to the first 20 readings in the csv file. Each row corresponds
            to a reading; the first 33 values are x-coordinates while the second33 values are y-coordinates
    """
    data = []
    with open(filename, 'r') as csvfile:
        r = csv.reader(csvfile, delimiter=' ')
        for row in r:
            data.append([float(i) for i in row[0].split(',')])
    return np.array(data)

In [ ]:
# Read the csv files using read_csv
# YOUR CODE HERE
genre1 = read_csv(...)
genre2 = read_csv(...)
genre3 = read_csv(...)
genre4 = read_csv(...)
...

# Take the same number of readings for both genres to be fair
num_samples = min(np.shape(genre1)[0], np.shape(genre2)[0], np.shape(genre3)[0], np.shape(genre4)[0]), ...
genre1 = genre1[:num_samples,:]
genre2 = genre2[:num_samples,:]
genre3 = genre3[:num_samples,:]
genre4 = genre4[:num_samples,:]
...

In [ ]:
# Plot the readings
plt.plot(genre1.T)
plt.figure()
plt.plot(genre2.T)
plt.figure()
plt.plot(genre3.T)
plt.figure()
plt.plot(genre4.T)
plt.show()
...

The plots that you just generated show the frequency spectrum of each of the samples you just collected. Each plot is a different genre and each line is a sample within that genre. 

<b>Note, that the spectrum looks mirrored across the center. This is due to sampling and aliasing that you will learn about soon. For now, what you need to know is that highest frequency content is in the middle and the second half of the data is redundant.</b>

<center>
<img width="400px" src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/proj-aliased-spectrum.png">
</center>

Start by looking at just one genre. Note that even samples from the same genre might look somewhat different. Because SIXT33N samples only a short section of music and then looks at the DFT, the music might look very different between samples. A sample might be taken at a loud instrumental part of the music or a quiet part when not much is happening. The important part to look at is what most of the samples seem to be doing at a particular frequency.

Now, compare the plots across genres. They should be pretty different. Does the genre have a lot of high frequency content? Which two genres do you think PCA will be able to tell apart the best? If you are having trouble telling apart the genres, PCA will probably have trouble doing so as well.

Pick two genres that you think PCA will classify the best, those are the genres that you will use for the rest of the code in this part. However, if your results are not what you would like, feel free to try another pairing.

You will now write the function <b>`process_data`</b> below that processes the raw data and puts it into a matrix that the Principal Component Analysis can use. Processing data is about taking raw, noisy data and removing noise and accentuating important information so that classification algorithms like PCA are working with the most information that we can give them. Experiment with a couple methods of signal processing.

- Try removing redundant information.
- Filtering is a good way of removing noise. Try experimenting with using different ranges of your spectrum. 
- You have data for both real and imaginary parts of your FFT, does one work better than the other? You might also want to think about ways to combine the real and imaginary parts of each coefficient and somehow normalize each row.

Keep in mind that the same data pre-processing algorithm has to run on the Launchpad, so try to keep it as simple as possible.

In [ ]:
def process_data(data):
    """
    Construct the A matrix so that it is friendly for PCA. This may include some data manipulation.
    Input:
        data: a Nx256 matrix containing 1 FFT data reading in each row. The real and imaginary
            parts of each coefficient are interleaved, so the first row contains
            [re0 im0 re1 im1 ... re127 im127] if reN is the real part of the Nth coefficient
            and imN is the imaginary part of the Nth coefficient
    Output:
        A: the A matrix for PCA
    """
    sA = np.shape(data)
    
    # YOUR CODE HERE
    A = 
    for i in range(len(data)):
        # YOUR CODE HERE
        # plt.plot(...) You can plot each row to see how your data processing changes the data
    return A

In [ ]:
# Build A matrix for PCA
A = process_data(genre1)
A = np.vstack((A, process_data(genre2)))


In [ ]:
# Examine the results of process_data() on the average across each genre
for i in range(num_samples//50):
    plt.plot(np.mean(A[i*50:(i+1)*50], 0), 'red')
for i in range(num_samples//50):
    plt.plot(np.mean(A[num_samples + i*50 : num_samples + (i+1)*50], 0), 'blue')

Try running the following SVD code on your matrix A, and get a base line of how well it works.

Once you get an idea of what it looks like on its own, you might want to experiment with using other types of pre-processing on your matrix A. For example, zero-meaning might have interesting results. Let processed_A be A with some signal processing applied to it.


**<span style="color:red">What processing techniques did you try on the matrix and and what was the result?</span>** 


In [ ]:
#YOUR CODE HERE
processed_A = A

Finally we are ready to take the SVD in order to retrieve the principal components. 

In [ ]:
# Take the SVD of processed_A
# YOUR CODE HERE
[u,s,v] = 

Now that we have taken the SVD, let's examine the results. Plot the sigma values. How many principle components do the sigma values suggest you will need? If the results of the SVD are not satisfactory, think of other ways you can change `processed_A` to get a better result. 

Once you feel that your sigma values are satisfactory, you will know how many principal components you need. (Note, if this number is not small, you might need to go back and try to get better results.)


In [ ]:
# Plot the sigma values
plt.stem(s)
plt.xlim([-0.5,10])
plt.title('Sigma values')

Take a look at the principle components you have chosen to use. What do they tell you about the important features for distinguishing between genres?

Then, project the data on the principal component(s) you found above and plot the projection. How well does the new basis separate the genres?

In [ ]:
# Plot the significant principal component
# YOUR CODE HERE


In [ ]:
# Project the data matrix to the first principal component
# YOUR CODE HERE
proj = 

# Plot the projection
x = np.linspace(0,num_samples, num_samples)
plt.scatter(x,proj[:num_samples], c=['red'], edgecolors='none', alpha=0.7)
plt.scatter(x + num_samples, proj[num_samples:], c=['blue'], edgecolors='none', alpha=0.7)
plt.legend(['genre1', 'genre2'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

The above code plots each sample, projected into the new basis. The x axis is the sample number. So the left half of the x axis is all the samples from the first genre in order, and the right half is all the samples from the other genre. If you used one principal component, then the y axis will be where PCA has clustered the sample. You should be able to draw a horizontal line in your head such that most of the points on the right are on one side of the line and most of the points on the left are on the other side of the line.

You can also try plotting a histogram of the projection results to see how each genre behaves with the PCA vectors. This is another way of looking at the same results by seeing how many points are classified where. You would like to see a red peak, and a blue peak, with little overlap.

In [ ]:
plt.hist(proj[:num_samples], 100, color=['red'], alpha=0.5, histtype='stepfilled')
plt.hist(proj[num_samples:], 100, color=['blue'], alpha=0.5, histtype='stepfilled')
plt.legend(['genre1', 'genre2'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

<img width='30px' align='left' src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/check.png">
<br/>
## <span style="color:green">CHECKPOINT 1</span>
 <span style="color:green">**First pass through PCA with sample data.** Show your GSI the result of the projection and talk about how you might be able to improve the result.</span>

Your data might look noisy, and might not classify perfectly. That is completely okay, we are just looking for good enough. Like many AI applications, this is very noisy data that we are classifying so some error in classification is okay. The important part is if you think that you can see some clustering. 

Once you think you have decent clustering, you can move on to getting your code to automate classification and you will make up for some of the error there, too. However, if you don't see clustering, go back and and see what you can do to improve it. If you cannot separate your data with some decent accuracy, your code probably cannot either.


<a id='part3'></a>
## <span style="color:blue">Part 3: Classification</span>

Using what you learned about your data in the last section, you will define a way of separating the genres. In the same way that you separated the data by eye, you will now write code to automate the process. Fill in the skeleton code that we have provided below. The function classify should take a data vector and a PC vector as input, <b>do all of the signal processing you decided on in the previous part</b>, then project the data into the new basis, and reference some threshold to decide which genre the data vector should be classified as.

Basically, you are taking all of the processing code from the previous part and putting it into a single function so that you can see how it performs.

In [ ]:
def classify(vector, new_basis):
    """
    Classifies a new reading vector into one of the 2 genres.
    Inputs:
        vector: 1xM reading vector - pre-processed data vector 
            (i.e. a line of A - NOT processed_A)
        new_basis: matrix with the basis of the new space
    Output:
        String of the classified genre
    """
    # YOUR CODE HERE
    proj = 
    if (...):
        return 'genre1'
    if (...):
        return 'genre2'

In [ ]:
# Try out the classification function
print(classify(A[0,:], ...)) # Modify to use other vectors

Next, you will check the accuracy of your classification. <b>Our target is for the classifier to classify at least 80% correctly on both genres.</b> Write code to apply the `classify` function to each row in `A` and check whether the classification is accurate. Compute the accuracy of classifying your first genre correctly and the accuracy of classifying the second genre correctly. These values should be about equal, and large differences in them might help you identify what needs to change in your <b>`classify`</b> function. Don't worry if the accuracy is not high enough - we will fix that soon.

In [ ]:
# Try to classify the whole A matrix
genre1_good = 0;
genre1_bad = 0;
genre2_good = 0;
genre2_bad = 0;

for row in A:
    # YOUR CODE HERE
    
print("Percent Correct of Genre 1 = ", ...)
print("Percent Correct of Genre 1 = ", ...)

Are the accuracy numbers high enough? Did you hit at least 80%? If the accuracy still is not as high, think of other ways to make the classifier better. 

Remember that SIXT33N has as long as it likes to make a decision before moving. Consider collecting multiple samples of music before deciding which genre the music belongs to, using some sort of majority function to choose the final answer.

In [ ]:
# Experiment with different methods to get the accuracy of classification 80% on both genres
# YOUR CODE HERE
genre1_good = 0;
genre1_bad = 0;
genre2_good = 0;
genre2_bad = 0;

for (i,row) in enumerate(A):
    # YOUR CODE HERE

<img width='30px' align='left' src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/check.png">
<br/>
## <span style="color:green">CHECKPOINT 2</span>
 <span style="color:green">**Show your GSI the classification that has at least 80% accuracy on both genres.**</span>

<a id='part4'></a>
## <span style="color:blue">Head Start: Launchpad Implementation</span>

### Materials
- Microphone front-end circuit
- Launchpad + USB

In the next phase you will collect your true data set on your teammate's microphone circuit, and then implement your <b>data processing</b> and <b>classification</b> on the Launchpad.

If you finish this phase within the first two weeks, then we strongly recommmend you get ahead by working on "Part 3:Launchpad Implementation" in `SIXT33N-processing-music.ipynb` to get a jump start on the processing integration phase. That section will walk you through taking the signal procesing that you just developed above and translating it from python to C that will go on your launchpad.

In addition, you will need to transfer the PCA vector and mean you found here into the launchpad. To do so, simply run the block below and copy each output into the Launchpad code.

You may also use this code for any other vectors that you need in your code.

In [ ]:
a = ''
for i in v[0,:].T:
    a += str(i) + ', '
print("PC1")
print(a)
print(" ")
